# Cloud Mask Validation Notebook

Notebook to visualize EVHR data and cloud masks while not having good WIFI to actually download the imagery. Also serves well to visualize data on the fly.

In [1]:
import os
import re
import tempfile
import folium
import numpy as np
import rasterio as rio
import ipywidgets as widgets
import branca.colormap as cm
import matplotlib.pyplot as plt
import pandas as pd
import altair as alt
import json

from glob import glob
from pathlib import Path
from folium import plugins
from pyproj import Transformer 
from rasterio.warp import calculate_default_transform, reproject, Resampling

os.environ['LOCALTILESERVER_CLIENT_PREFIX'] = \
    f"{os.environ['JUPYTERHUB_SERVICE_PREFIX'].lstrip('/')}/proxy/{{port}}"

from localtileserver import get_folium_tile_layer, TileClient

Define data paths and regex.

In [2]:
"""
input_bands:
- CoastalBlue
- Blue
- Green
- Yellow
- Red
- RedEdge
- NIR1
- NIR2
"""
data_bands = [5, 7, 2]
data_dir = '/adapt/nobackup/projects/ilab/projects/srlite/input/Serc/*-toa.tif'
mask_dir = '/adapt/nobackup/projects/ilab/projects/CloudMask/products/srlite/v1.2/Serc/*.cloudmask.tif'

Get list of filenames.

In [3]:
data_filenames = sorted(glob(data_dir))
mask_filenames = sorted(glob(mask_dir))
assert len(data_filenames) == len(mask_filenames), \
    'Not all layers have the same number of file'
print(f'Loading {len(data_filenames)} filenames.')

Loading 40 filenames.


In [4]:
# ONLY NEEDED IF YOU WANT TO GROUP FEATURES INTO LAYERS

filenames_metadata = dict()
for d, m, in zip(data_filenames, mask_filenames):
    
    #raster_date = re.search(r'\d{4}\d{2}\d{2}', d).group(0)
    raster_date = re.search(r'\d{4}', d).group(0)
    
    #print(filenames_metadata.keys())
    if raster_date not in filenames_metadata:
        filenames_metadata[raster_date] = [{'data_layer': d, 'mask_layer': m}]
    else:
        filenames_metadata[raster_date].append({'data_layer': d, 'mask_layer': m})
    #print(raster_date.group(0), raster_id.group(0), d)


filenames_metadata = dict(sorted(filenames_metadata.items()))

Here we define basemaps for use with folium on the backend.

In [5]:
basemaps = {
       'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = False,
        control = True
       ),
        'basemap_gray': folium.TileLayer(
            tiles="http://services.arcgisonline.com/ArcGIS/rest/services/Canvas/World_Light_Gray_Base/MapServer/tile/{z}/{y}/{x}",
            opacity=1,
            name="World gray basemap",
            attr="ESRI",
            overlay=False
        ),
        'Imagery': folium.TileLayer(
            tiles='https://services.arcgisonline.com/arcgis/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
            opacity=1,
            name="World Imagery",
            attr="ESRI",
            overlay=False
        ),
        'ESRINatGeo': folium.TileLayer(
            tiles='https://server.arcgisonline.com/ArcGIS/rest/services/NatGeo_World_Map/MapServer/tile/{z}/{y}/{x}',
            opacity=1,
            name='ESRI NatGeo',
            attr='ESRI',
            overlay=False
        )
}

Define the entire folium layout.

In [6]:
# Client - initial client to localize zoom
data_client = TileClient(data_filenames[0])

# Map the Layers
folium_map = folium.Map(
    location=data_client.center(),
    zoom_start=16,
    tiles='https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
    attr='Google'
)

# Iterate over each item for specific dates
for date_key in filenames_metadata:

    print(f'Processed year: {date_key}')
    
    # Divide feature groups into year
    feature_group_data = folium.FeatureGroup(name=f'EVHR - {date_key}', show=False)
    feature_group_mask = folium.FeatureGroup(name=f'CloudMask  - {date_key}', show=False)

    # iterate over data files
    for raster_id in filenames_metadata[date_key]:
        
        # client for data tile
        data_client = TileClient(raster_id['data_layer'])
    
        # client for landcover tile
        mask_client = TileClient(raster_id['mask_layer'])
    
        # group all data layers into one single control
        feature_group_data.add_child(
            get_folium_tile_layer(data_client, show=False))
    
        # group all landcover layers into one single control
        feature_group_mask.add_child(
            get_folium_tile_layer(mask_client, cmap='Greens', show=False))
    
        # add a marker per location
        #folium.Marker(
        #    data_client.center(),
        #    popup=re.search(r'Tappan\d{2}', raster_id['data_layer']).group(0)
        #).add_to(folium_map)
    
    folium_map.add_child(feature_group_data)
    folium_map.add_child(feature_group_mask)

folium_map.add_child(plugins.Fullscreen())
folium_map.add_child(plugins.Geocoder())
folium_map.add_child(plugins.MousePosition())
folium_map.add_child(folium.LayerControl())

folium_map

Processed year: 2010
Processed year: 2011
Processed year: 2015
Processed year: 2016
Processed year: 2017
Processed year: 2018
Processed year: 2019
Processed year: 2020
Processed year: 2021
